In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ipl-complete-dataset-20082020/IPL Matches 2008-2020.csv
/kaggle/input/ipl-complete-dataset-20082020/IPL Ball-by-Ball 2008-2020.csv


<font size="+3" color=red><b> <center><u> Fantasy IPL Dream Team🏏 for whole tournament and season 2020 </u></center></b></font>

In [2]:
df1=pd.read_csv("/kaggle/input/ipl-complete-dataset-20082020/IPL Matches 2008-2020.csv")
df1.head()

,id,city,date,player_of_match,venue,neutral_venue,team1,team2,toss_winner,toss_decision,winner,result,result_margin,eliminator,method,umpire1,umpire2
0,335982,Bangalore,2008-04-18,BB McCullum,M Chinnaswamy Stadium,0,Royal Challengers Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,field,Kolkata Knight Riders,runs,140.0,N,NaN,Asad Rauf,RE Koertzen
1,335983,Chandigarh,2008-04-19,MEK Hussey,"Punjab Cricket Association Stadium, Mohali",0,Kings XI Punjab,Chennai Super Kings,Chennai Super Kings,bat,Chennai Super Kings,runs,33.0,N,NaN,MR Benson,SL Shastri
2,335984,Delhi,2008-04-19,MF Maharoof,Feroz Shah Kotla,0,Delhi Daredevils,Rajasthan Royals,Rajasthan Royals,bat,Delhi Daredevils,wickets,9.0,N,NaN,Aleem Dar,GA Pratapkumar
3,335985,Mumbai,2008-04-20,MV Boucher,Wankhede Stadium,0,Mumbai Indians,Royal Challengers Bangalore,Mumbai Indians,bat,Royal Challengers Bangalore,wickets,5.0,N,NaN,SJ Davis,DJ Harper
4,335986,Kolkata,2008-04-20,DJ Hussey,Eden Gardens,0,Kolkata Knight Riders,Deccan Chargers,Deccan Chargers,bat,Kolkata Knight Riders,wickets,5.0,N,NaN,BF Bowden,K Hariharan


In [3]:
df2=pd.read_csv("/kaggle/input/ipl-complete-dataset-20082020/IPL Ball-by-Ball 2008-2020.csv")
df2.head()

,id,inning,over,ball,batsman,non_striker,bowler,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,batting_team,bowling_team
0,335982,1,6,5,RT Ponting,BB McCullum,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
1,335982,1,6,6,BB McCullum,RT Ponting,AA Noffke,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
2,335982,1,7,1,BB McCullum,RT Ponting,Z Khan,0,0,0,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
3,335982,1,7,2,BB McCullum,RT Ponting,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore
4,335982,1,7,3,RT Ponting,BB McCullum,Z Khan,1,0,1,0,0,NaN,NaN,NaN,NaN,Kolkata Knight Riders,Royal Challengers Bangalore


In [4]:
# We need two dataframes one is for whole tournament and other is for year 2020
# Merge df1 and df2(onle two columns(id,date)) on 'id'
data = pd.merge(df2, df1[['id','date']], on='id',how='left')

# Extract year from date column
data['year'] = pd.to_datetime(data['date']).dt.year
data.drop('date',axis=1,inplace=True)

# This dataframe is only for year 2020
season_2020 = data[data['year']==2020]
# This is basically copy of df2 because it contains data of whole tournament
tournament = df2.copy()

# ****Batting Points****

 **Run                       -> +1**
 
 **Boundary Bonus            -> +1**
 
 **Six Bonus                 -> +2**
 
 **30 Run Bonus              -> +4**
 
 **Half-century Bonus        -> +8**
 
 **Century Bonus             -> +16**

In [5]:
# It finds bat_score according to point system mentioned above.
def batsman_score(x):
    x["bat_score"]=1*x["runs"]+1*x["fours"]+2*x["sixes"]+4*x["thirty"]+8*x["fifty"]+16*x["century"]
    return x

def get_bat_stats(df):
    # Initialize batting dataframe.
    batting_df = pd.DataFrame(columns=["player","runs","fours","sixes","thirty","fifty","century"])

    # Let's use groupby function. Our aim is to find batting score so we'll group our data by batsman.
    for player,details in df.groupby("batsman"):
        total_runs = sum(details["batsman_runs"])

        # If batsman_runs is 4 then we count it as four.
        fours=0
        try:
            fours=details["batsman_runs"].value_counts()[4]
        except:
            fours=0      

        # If batsman_runs is 6 then we count it as six.
        sixes=0
        try:
            sixes=details["batsman_runs"].value_counts()[6]
        except:
            sixes=0 

        thirty=0
        fifty=0
        century=0

        # To find thirty, fifty, century of the player we need to group our dataframe(details) by match id.
        for match,match_details in details.groupby("id"):

            # Find total score of the player by sum up total runs.
            match_score = sum(match_details["batsman_runs"])

            # Accourding to match_score this count thirties, fifties, centuries.
            if match_score>=100:
                century+=1
            elif match_score>=50:
                fifty+=1
            elif match_score>=30:
                thirty+=1

        # Store all the data into batting dataframe.
        batting_df = batting_df.append({"player":player,"runs":total_runs,"fours":fours,"sixes":sixes,"thirty":thirty,"fifty":fifty,"century":century},ignore_index=True)
    
    # To find bat_score we apply batsman_score function
    batting_df=batting_df.apply(batsman_score,axis=1)
    return batting_df

# ****Bowling Points****

 **Wicket                  -> +25**
 
 **Bonus (LBW / Bowled)    -> +8**
 
 **3 Wicket Bonus          -> +4**
 
 **4 Wicket Bonus          -> +8**
 
 **5 Wicket Bonus          -> +16**

In [6]:
# Same as bat_score here we find bowl_score through point system mentioned above.
def bowler_score(x):
    x["bowl_score"]=25*x["wickets"]+8*x["lbw_or_bowled"]+4*x["three_wickets"]+8*x["four_wickets"]+16*x["five_wickets"]
    return x

def get_bowl_stats(df):
    # Initialize bowling dataframe.
    bowling_df = pd.DataFrame(columns=["player","wickets","lbw_or_bowled","three_wickets","four_wickets","five_wickets"])

    # Let's use groupby function. Our aim is to find bowling score so we'll group our data by bowler.
    for player,details in df.groupby("bowler"):

        # It finds the length of dataframe where dismissal kind is either lbw or bowled
        lbw_or_bowled=len(details.query('dismissal_kind == "lbw" or dismissal_kind == "bowled"'))

        total_wickets=0
        three_wickets=0
        four_wickets=0
        fifer=0

        # To find three wicket haul, four wicket haul and five wicket haul we should group dataframe(details) again by match id.
        for match,match_details in details.groupby("id"):
            wickets = len(match_details.query('dismissal_kind in ["caught","bowled","lbw","stumped","caught and bowled"]'))
            if wickets>=3:
                three_wickets+=1
            elif wickets>=4:
                four_wickets+=1
            elif wickets>=5:
                fifer+=1
            # Need to add every match wickets into total_wickets to find total wickets of bowler.
            total_wickets+=wickets

        # Store all the data into bowling dataframe.
        bowling_df = bowling_df.append({"player":player,"wickets":total_wickets,"lbw_or_bowled":lbw_or_bowled,"three_wickets":three_wickets,"four_wickets":four_wickets,"five_wickets":fifer},ignore_index=True)
        
    # To find bowl_score we apply bowler_score function.
    bowling_df=bowling_df.apply(bowler_score,axis=1) 
    return bowling_df

In [7]:
def get_wicketkeepers(df):
    # Let's find out wicketkeepers from this dataset. To achieve this we'll check dismissal kind, if it is stumpout then store such fielders name into list
    return list(df.query('dismissal_kind == "stumped"')['fielder'].unique())

**Our Dream11 team will have different combinations of players...**

**Batsman (BAT) - 4**

**Wicket Keeper (WK) - 1**

**All Rounder (AR) - 3**

**Bowler (BWL) - 3**

# **For whole Tournament**

In [8]:
batsmen_tournament = get_bat_stats(tournament)
batsmen_tournament.head()

,player,runs,fours,sixes,thirty,fifty,century,bat_score
0,A Ashish Reddy,280,16,15,2,0,0,334
1,A Chandila,4,0,0,0,0,0,4
2,A Chopra,53,7,0,0,0,0,60
3,A Choudhary,25,1,1,0,0,0,28
4,A Dananjaya,4,0,0,0,0,0,4


In [9]:
bowlers_tournament = get_bowl_stats(tournament)
bowlers_tournament.head()

,player,wickets,lbw_or_bowled,three_wickets,four_wickets,five_wickets,bowl_score
0,A Ashish Reddy,18,9,1,0,0,526
1,A Chandila,11,0,1,0,0,279
2,A Choudhary,5,0,0,0,0,125
3,A Dananjaya,0,0,0,0,0,0
4,A Flintoff,2,0,0,0,0,50


In [10]:
merged_data_tournament = pd.merge(batsmen_tournament,bowlers_tournament,on="player",how="outer").fillna(0)
merged_data_tournament.head()

,player,runs,fours,sixes,thirty,fifty,century,bat_score,wickets,lbw_or_bowled,three_wickets,four_wickets,five_wickets,bowl_score
0,A Ashish Reddy,280.0,16.0,15.0,2.0,0.0,0.0,334.0,18.0,9.0,1.0,0.0,0.0,526.0
1,A Chandila,4.0,0.0,0.0,0.0,0.0,0.0,4.0,11.0,0.0,1.0,0.0,0.0,279.0
2,A Chopra,53.0,7.0,0.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0
3,A Choudhary,25.0,1.0,1.0,0.0,0.0,0.0,28.0,5.0,0.0,0.0,0.0,0.0,125.0
4,A Dananjaya,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Find allrounder score
merged_data_tournament['allrounder_score']=(merged_data_tournament['bat_score']*merged_data_tournament['bowl_score'])/1000
merged_data_tournament.head()

,player,runs,fours,sixes,thirty,fifty,century,bat_score,wickets,lbw_or_bowled,three_wickets,four_wickets,five_wickets,bowl_score,allrounder_score
0,A Ashish Reddy,280.0,16.0,15.0,2.0,0.0,0.0,334.0,18.0,9.0,1.0,0.0,0.0,526.0,175.684
1,A Chandila,4.0,0.0,0.0,0.0,0.0,0.0,4.0,11.0,0.0,1.0,0.0,0.0,279.0,1.116
2,A Chopra,53.0,7.0,0.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
3,A Choudhary,25.0,1.0,1.0,0.0,0.0,0.0,28.0,5.0,0.0,0.0,0.0,0.0,125.0,3.500
4,A Dananjaya,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000


In [12]:
wicketkeeper_tournament = get_wicketkeepers(tournament)

In [13]:
# According to score in particular role we sort data and store players name into dictionary. This dictonary contains players name and players role.
total_batsmen = 4
total_bowlers = 3
total_allrounders = 3
total_wicketkeepers = 1
playing_XI_tournament = {}

playing_XI_tournament['BAT'] = merged_data_tournament.sort_values(by="bat_score",ascending=False).head(total_batsmen)['player'].to_list()
playing_XI_tournament['WK'] = merged_data_tournament.query('player in @wicketkeeper_tournament').sort_values(by="bat_score",ascending=False).head(total_wicketkeepers)['player'].to_list()
playing_XI_tournament['AR'] = merged_data_tournament.sort_values(by="allrounder_score",ascending=False).head(total_allrounders)['player'].to_list()
playing_XI_tournament['BWL'] = merged_data_tournament.sort_values(by="bowl_score",ascending=False).head(total_bowlers)['player'].to_list()

In [14]:
print('Fantasy dream team for the whole tournament\n')
for player_role,players in playing_XI_tournament.items():
    for player in players:
        print("{:15} {}".format(player,player_role))

Fantasy dream team for the whole tournament

V Kohli         BAT
SK Raina        BAT
DA Warner       BAT
RG Sharma       BAT
AB de Villiers  WK
SR Watson       AR
RA Jadeja       AR
DJ Bravo        AR
SL Malinga      BWL
PP Chawla       BWL
A Mishra        BWL


# **For Season 2020**

In [15]:
batsmen_2020 = get_bat_stats(season_2020)
batsmen_2020.head()

,player,runs,fours,sixes,thirty,fifty,century,bat_score
0,A Nortje,7,0,0,0,0,0,7
1,AB de Villiers,454,33,23,2,5,0,581
2,AD Russell,117,9,9,0,0,0,144
3,AJ Finch,268,28,8,2,1,0,328
4,AJ Tye,6,0,1,0,0,0,8


In [16]:
bowlers_2020 = get_bowl_stats(season_2020)
bowlers_2020.head()

,player,wickets,lbw_or_bowled,three_wickets,four_wickets,five_wickets,bowl_score
0,A Mishra,3,0,0,0,0,75
1,A Nortje,22,9,2,0,0,630
2,A Zampa,2,0,0,0,0,50
3,AD Russell,5,1,0,0,0,133
4,AJ Tye,1,0,0,0,0,25


In [17]:
merged_data_2020 = pd.merge(batsmen_2020,bowlers_2020,on="player",how="outer").fillna(0)
merged_data_2020.head()

,player,runs,fours,sixes,thirty,fifty,century,bat_score,wickets,lbw_or_bowled,three_wickets,four_wickets,five_wickets,bowl_score
0,A Nortje,7.0,0.0,0.0,0.0,0.0,0.0,7.0,22.0,9.0,2.0,0.0,0.0,630.0
1,AB de Villiers,454.0,33.0,23.0,2.0,5.0,0.0,581.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AD Russell,117.0,9.0,9.0,0.0,0.0,0.0,144.0,5.0,1.0,0.0,0.0,0.0,133.0
3,AJ Finch,268.0,28.0,8.0,2.0,1.0,0.0,328.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AJ Tye,6.0,0.0,1.0,0.0,0.0,0.0,8.0,1.0,0.0,0.0,0.0,0.0,25.0


In [18]:
# Find allrounder score
merged_data_2020['allrounder_score']=(merged_data_2020['bat_score']*merged_data_2020['bowl_score'])/1000
merged_data_2020.head()

,player,runs,fours,sixes,thirty,fifty,century,bat_score,wickets,lbw_or_bowled,three_wickets,four_wickets,five_wickets,bowl_score,allrounder_score
0,A Nortje,7.0,0.0,0.0,0.0,0.0,0.0,7.0,22.0,9.0,2.0,0.0,0.0,630.0,4.410
1,AB de Villiers,454.0,33.0,23.0,2.0,5.0,0.0,581.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
2,AD Russell,117.0,9.0,9.0,0.0,0.0,0.0,144.0,5.0,1.0,0.0,0.0,0.0,133.0,19.152
3,AJ Finch,268.0,28.0,8.0,2.0,1.0,0.0,328.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
4,AJ Tye,6.0,0.0,1.0,0.0,0.0,0.0,8.0,1.0,0.0,0.0,0.0,0.0,25.0,0.200


In [19]:
wicketkeeper_2020 = get_wicketkeepers(season_2020)

In [20]:
# According to score in particular role we sort data and store players name into dictionary. This dictonary contains players name and players role.
total_batsmen = 4
total_bowlers = 3
total_allrounders = 3
total_wicketkeepers = 1
playing_XI_2020 = {}

playing_XI_2020['BAT'] = merged_data_2020.sort_values(by="bat_score",ascending=False).head(total_batsmen)['player'].to_list()
playing_XI_2020['WK'] = merged_data_2020.query('player in @wicketkeeper_2020').sort_values(by="bat_score",ascending=False).head(total_wicketkeepers)['player'].to_list()
playing_XI_2020['AR'] = merged_data_2020.sort_values(by="allrounder_score",ascending=False).head(total_allrounders)['player'].to_list()
playing_XI_2020['BWL'] = merged_data_2020.sort_values(by="bowl_score",ascending=False).head(total_bowlers)['player'].to_list()

In [21]:
print('Fantasy dream team for season 2020\n')
for player_role,players in playing_XI_2020.items():
    for player in players:
        print("{:15} {}".format(player,player_role))

Fantasy dream team for season 2020

KL Rahul        BAT
S Dhawan        BAT
DA Warner       BAT
Ishan Kishan    BAT
Q de Kock       WK
MP Stoinis      AR
R Tewatia       AR
SM Curran       AR
K Rabada        BWL
JJ Bumrah       BWL
TA Boult        BWL
